# 4. Run SMBalance Module

# 4.1 Import Functions

In [ ]:
import os
import glob
#import WAPORWA modules
os.chdir(r'C:\WIT_Training_material\Zarqa_WA\WAPORWA-master\modules') #change to modules path
import WA
from WA.pickle_basin import pickle_in,pickle_out  
from WA.model_SMBalance import run_SMBalance
from dask.distributed import Client

#Read pickle
Main_dir=r"C:\WIT_Training_material\Zarqa_WA\Main"
pickle=glob.glob(os.path.join(Main_dir,'*.pickle'))[-1] 
BASIN=pickle_in(pickle)  

#Customize dask performance
client = Client()
client

In [ ]:
BASIN

# 4.2 Run SM Balance

In [ ]:
import  datetime
starttime = datetime.datetime.now()


## Read model input data from pickled dictionary
MAIN_FOLDER= os.path.join(BASIN['Dir'],'data','nc')


## read netCDF files    
smsat_file = BASIN['main_data']['stat']['thetasat'] #saturated soil moisture content
p_in = BASIN['main_data']['monthly']['p'] # monthly precipitation
e_in = BASIN['main_data']['monthly']['et'] # monthly actual evapotranspiration
i_in = BASIN['main_data']['monthly']['i'] # monthly interception
rd_in = BASIN['main_data']['monthly']['nRD'] # monthly number of rainy days
lu_in = BASIN['main_data']['yearly']['lcc']  # monthly land cover class

outputs=run_SMBalance(MAIN_FOLDER,
                      p_in, e_in, i_in, rd_in, lu_in, smsat_file,                      
                      f_perc=1, # percolation factor
                      f_Smax=0.9, # threshold percolation parameter
                      cf =  20, # f_Ssat soil mositure correction factor to componsate the variation in filling up and drying in a month
                      chunks=[1,300,300] # chunk size [time,lon, lat]
                     )

BASIN['main_data']['monthly']['etrain']=outputs[0]
BASIN['main_data']['monthly']['etincr']=outputs[1]
pickle_out(BASIN)



endtime = datetime.datetime.now()
print(endtime-starttime)